In [1]:
from datetime import date
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
URL = 'https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end=20200503'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
results= soup.find('div', class_='cmc-table__table-wrapper-outer').find('thead')
#for result in results:
    #btc_price = result
#btc_price
#results

In [3]:
clounms_name=[th.text for th in results.tr.find_all('th')]
clounms_name

['Date', 'Open*', 'High', 'Low', 'Close**', 'Volume', 'Market Cap']

In [ ]:
rows_d=soup.find_all('tr', class_='cmc-table-row')
#rows_d

In [ ]:
data_rows=[]
for tr in rows_d:
    rows=[td.text for td in tr.find_all('td')]
    data_rows.append(rows)
data_rows[0]

In [ ]:
df =pd.DataFrame(data_rows,columns=clounms_name)

del df['High']
data=df[['Date','Open*']]
data.head()

In [ ]:
data1=pd.DataFrame(columns=['Date','Open*'])
for x in range(len(data)):
    data1=data1.append(data.iloc[-(x+1)])
data1=data1.reset_index(drop=True)
data1.head()

In [ ]:
data1['Open*']=data1['Open*'].str.replace(',','').astype(float)
data1.loc[1]['Open*']

In [ ]:
increase=[0]
for i in range(1,len(data1)):
    si_increase=(data1.loc[i]['Open*']-data1.loc[i-1]['Open*'])/data1.loc[i-1]['Open*']*100
    #si_increase=format(si_increase,'.2%')  #按百分比数据列示
    #si_increase="%.2f%%"%(si_increase*100)
    si_increase=si_increase.round(4)
    increase.append(si_increase)
increase[0:8]
#Single_day_Increase="%.2f%%"%(increase*100)
#Single_day_Increase

In [ ]:
data1['Single_day_Increase(%)']=increase
data1.head()

In [ ]:

#single_in=data1['Single_day_Increase'].str.replace('%','').astype(float)/100
#single_in

In [ ]:
sixty_days_increase = [0.00]*59
for i in range(59,len(data1)):
    sixty_days_in=data1.loc[i-59:i]['Single_day_Increase(%)']
    sixty_days_in=sum(sixty_days_in)
    sixty_days_increase.append(sixty_days_in)
sixty_days_increase
data1['Sixty_Days_Increase(%)']=sixty_days_increase
data1.head(-5)

In [ ]:
data1['Date']=data1['Date'].str.replace(',','')
data1['Date']=pd.to_datetime(data1['Date'])
data1.loc[0]['Date']

In [ ]:
data1['Date']=data1['Date'].dt.date
data1.loc[0]['Date']

In [ ]:
data1.head()

In [ ]:
data1.to_csv('BTC_price_history.csv')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data_plt=data1[['Date','Sixty_Days_Increase(%)']]
data_plt=data_plt[59:]
#data_plt.plot(figsize = (20,10))
data_plt.plot(kind='line', x='Date', y='Sixty_Days_Increase(%)', ax= plt.gca(), grid=True,figsize = (20,10))